In [1]:
import pandas as pd
import numpy as np
import re
import glob
import matplotlib.pyplot as plt

%matplotlib inline

# Import data

In [2]:
# Read all txt files into a panda dataframe
def importData():
   
    path='../input/data-science-for-good-city-of-los-angeles/cityofla/CityofLA/Job Bulletins/*.txt'
    files=glob.glob(path)
    jobs_list=[]
    file_names=[]
    for file in files:
        with open(file,'r',errors='replace') as f:
            jobs_list.append(f.read())
        match = re.search('Bulletins/(.*\.txt)',file)
        file_names.append(match.group(1))
    jobs_df = pd.DataFrame({"File_Name":file_names,"job_info":jobs_list})
    
    return jobs_df

# Extract data using regex

In [3]:
# Define functions for pandas dataframe apply

# Get job title (JOB_CLASS_TITLE)
def get_job_title(text):
    title = text.split('\n',1)[0]
    return title.strip()

# Get class code (JOB_CLASS_NO)
def get_class_code(text):
    match = re.search(r'Class\s*Code:\s*(\d+)',text)
    if match:
        return match.group(1)
    else:
        return None

    
# Get open date (OPEN_DATE)
def get_open_date(text):
    match = re.search(r'Open\s*Date:\s*(\d+-\d+-\d+)',text)
    if match:
        return match.group(1)
    else:
        return None

# Get exam type
def get_exam_type(text):
    match = re.search(r'(?s)Open\s*Date(.*?)ANNUAL\s*SALARY',text)
    if match:
        result = match.group(1)
        op=re.search(r'open',result,flags=re.IGNORECASE)
        open_int_prom=re.search(r'(open.*comp.*?tive)|(comp.*?tive.*open)',result,flags=re.IGNORECASE)
        int_dept_prom=re.search(r'inter.*?mental',result,flags=re.IGNORECASE)
        dept_prom=re.search(r'dep.*?mental',result,flags=re.IGNORECASE)
        if int_dept_prom:
            return 'INT_DEPT_PROM'
        elif dept_prom:
            return 'DEPT_PROM'
        elif open_int_prom:
            return 'OPEN_INT_PROM'
        elif op:
            return 'OPEN'
        else:
            return None
    else:
        return None
    
# Get general salary (ENTRY_SALARY_GEN)
def get_salary_gen(text):
    match = re.search(r'(?s)ANNUAL\s*SALARY(.*?)DUTIES.*',text)
    if match:
        result = match.group(1)
        sal=re.search(r'(\d+,\d+.*?to.*?\d+,\d+|\d+,\d+).*?Water.*Power',text,flags=re.IGNORECASE)\
                if get_salary_dwp(text) else re.search(r'(\d+,\d+.*?to.*?\d+,\d+|\d+,\d+)',text,flags=re.IGNORECASE)
        if sal:
            return sal.group(1).replace('to','-').replace('$','')
        else:
            return None
    else:
        return None


# Get DWP salary (ENTRY_SALARY_DWP)
def get_salary_dwp(text):
    match = re.search(r'(?s)ANNUAL\s*SALARY(.*?)DUTIES.*',text)
    if match:
        result = match.group(1)
        sal=re.search(r'Water.*Power.*?(\d+,\d+.*?to.*?\d+,\d+|\d+,\d+)',text)
        if sal:
            return sal.group(1).replace('$','').replace('to','-')
        else:
            return None 
    else:
        return None
    
# Get driver license req (DRIVERS_LICENSE_REQ)
def get_dl_req(text):
    match= re.search(r"(.*?)driver\'s license",text)    
    if match:
        result = match.group(1).lower().split()
        if 'may' in result:
            return 'P'
        else:
            return 'R'
    else:
        return None
    
# Get driver license type (DRIV_LIC_TYPE)
def get_dl_type(text):
    dl_types=[]
    match= re.search(r"(?s)(valid California Class|valid Class|valid California Commercial Class)(.*?)(California driver\'s license|driver\'s license)",text)
    if match:
        dl=match.group(2)
        if 'A' in dl:
            dl_types.append('A')
        if 'B' in dl:
            dl_types.append('B') 
        if 'C' in dl:
            dl_types.append('C')  
        if 'I' in dl:
            dl_types.append('I')   
        return ','.join(dl_types)
    else:
        return None

# Get duties (JOB_DUTIES)
def get_duties(text):
    match= re.search(r"(?s)DUTIES(.*?)(REQ.*?MENT|MINI.*?REQ)", text)
    if match:
        return match.group(1).strip()
    else:
        return None

# Get requirements section
def get_req_section(text):  
    match= re.search(r"(?s)(QUAL.*?TIONS*|REQ.*?MENTS*).*?\n(.*?)(PROCESS NOTES|NOTES|WHERE TO APPLY|HOW TO APPLY)", text)
    if match:
        return match.group(2).strip()
    else:
        return None

# Split requirements to rows
def split_req(df):
    req_list = df['Req_section'].apply(lambda x: re.split(r'or\s*\n+(?=\d\.)', x))
    df = pd.DataFrame({col:np.repeat(df[col].values, req_list.str.len()) \
                  for col in df.columns})
    df['req_list'] = np.concatenate(req_list.values)

    return df
    
    
# Extract all by regex
def extract_df(df):    
    # JOB_CLASS_TITLE
    df['JOB_CLASS_TITLE'] = df['job_info'].apply(lambda x: get_job_title(x))
    # JOB_CLASS_NO
    df['JOB_CLASS_NO'] = df['job_info'].apply(lambda x: get_class_code(x))
    # OPEN_DATE
    df['OPEN_DATE'] = df['job_info'].apply(lambda x: get_open_date(x))
    # EXAM_TYPE
    df['EXAM_TYPE'] = df['job_info'].apply(lambda x: get_exam_type(x))
    # ENTRY_SALARY_GEN
    df['ENTRY_SALARY_GEN'] = df['job_info'].apply(lambda x: get_salary_gen(x))
    # ENTRY_SALARY_DWP
    df['ENTRY_SALARY_DWP'] = df['job_info'].apply(lambda x: get_salary_dwp(x))
    # JOB_DUTIES
    df['JOB_DUTIES'] = df['job_info'].apply(lambda x: get_duties(x))
    # DRIVERS_LICENSE_REQ
    df['DRIVERS_LICENSE_REQ'] = df['job_info'].apply(lambda x: get_dl_req(x))
    # DRIV_LIC_TYPE
    df['DRIV_LIC_TYPE'] = df['job_info'].apply(lambda x: get_dl_type(x))
    # Requirements section
    df['Req_section'] = df['job_info'].apply(lambda x: get_req_section(x))
    # Split reqs
    df = split_req(df)
    # Add req_set_id
    df['REQUIREMENT_SET_ID'] = df.groupby('JOB_CLASS_NO').cumcount() + 1
    
    return df

In [4]:
# Perform the extraction
df = importData()
df = extract_df(df)
df.head(10)

,File_Name,job_info,JOB_CLASS_TITLE,JOB_CLASS_NO,OPEN_DATE,EXAM_TYPE,ENTRY_SALARY_GEN,ENTRY_SALARY_DWP,JOB_DUTIES,DRIVERS_LICENSE_REQ,DRIV_LIC_TYPE,Req_section,req_list,REQUIREMENT_SET_ID
0,PARK MAINTENANCE SUPERVISOR 3145 102618.txt,PARK MAINTENANCE SUPERVISOR\n\nClass Code: ...,PARK MAINTENANCE SUPERVISOR,3145,10-26-18,OPEN,None,"83,728 - 88,405",A Park Maintenance Supervisor directly supervi...,R,None,1. Two years of full-time paid experience as a...,1. Two years of full-time paid experience as a...,1
1,PARK MAINTENANCE SUPERVISOR 3145 102618.txt,PARK MAINTENANCE SUPERVISOR\n\nClass Code: ...,PARK MAINTENANCE SUPERVISOR,3145,10-26-18,OPEN,None,"83,728 - 88,405",A Park Maintenance Supervisor directly supervi...,R,None,1. Two years of full-time paid experience as a...,2. One year full-time paid experience as noted...,2
2,ENVIRONMENTAL ENGINEER 7872 082616 REV 090116...,ENVIRONMENTAL ENGINEER \n\nClass Code: 7...,ENVIRONMENTAL ENGINEER,7872,08-26-16,OPEN,"89,637 - 127,555",None,An Environmental Engineer performs responsible...,R,None,Two years of full-time paid professional envir...,Two years of full-time paid professional envir...,1
3,MOTION PICTURE AND TELEVISION MANAGER 1789 111...,MOTION PICTURE AND TELEVISION MANAGER\n\n\nCla...,MOTION PICTURE AND TELEVISION MANAGER,1789,11-17-17,OPEN,"120,080 - 175,559",None,"A Motion Picture and Television Manager plans,...",R,None,1. Graduation from an accredited four-year col...,1. Graduation from an accredited four-year col...,1
4,HOUSING INVESTIGATOR 8516 062918.txt,HOUSING INVESTIGATOR\n\nClass Code: 8516...,HOUSING INVESTIGATOR,8516,06-29-18,OPEN,"59,487 - 86,965",None,A Housing Investigator conducts preliminary an...,R,None,1. Graduation from an accredited four-year col...,1. Graduation from an accredited four-year col...,1
5,HOUSING INVESTIGATOR 8516 062918.txt,HOUSING INVESTIGATOR\n\nClass Code: 8516...,HOUSING INVESTIGATOR,8516,06-29-18,OPEN,"59,487 - 86,965",None,A Housing Investigator conducts preliminary an...,R,None,1. Graduation from an accredited four-year col...,2. A paralegal certificate from a paralegal tr...,2
6,HOUSING INVESTIGATOR 8516 062918.txt,HOUSING INVESTIGATOR\n\nClass Code: 8516...,HOUSING INVESTIGATOR,8516,06-29-18,OPEN,"59,487 - 86,965",None,A Housing Investigator conducts preliminary an...,R,None,1. Graduation from an accredited four-year col...,3. Four years of full-time paid experience inv...,3
7,DEPARTMENTAL CHIEF ACCOUNTANT 1593 111717 revi...,DEPARTMENTAL CHIEF ACCOUNTANT\n\n ...,DEPARTMENTAL CHIEF ACCOUNTANT,1593,11-17-17,OPEN,"91,913 - 130,771",None,A Departmental Chief Accountant plans and dire...,None,None,Two years of full-time paid professional accou...,Two years of full-time paid professional accou...,1
8,POLICE LIEUTENANT 2232 020918.txt,POLICE LIEUTENANT\n Class Code: 2232 ...,POLICE LIEUTENANT,2232,02-09-18,OPEN,"125,572 - 139,916",None,"A Police Lieutenant plans, organizes, and dire...",None,None,"1. Two years as a Police Sergeant, Police Dete...","1. Two years as a Police Sergeant, Police Dete...",1
9,AUDITOR 1517 031816 (1).txt,AUDITOR\nClass Code: 1517\nOpen Date: ...,AUDITOR,1517,03-18-16,OPEN,"54,455 - 79,594",None,An Auditor conducts audits of financial and in...,P,None,1. One year of full-time paid professional exp...,1. One year of full-time paid professional exp...,1


# Extract data using NER in spaCy

## Generate spacy training data

In [5]:
import logging
import json
# Convert json to spacy training data structure
def json_to_spacy(filePath):
    try:
        training_data = []
        lines=[]
        with open(filePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + filePath + "\n" + "error = " + str(e))
        return None
    
# Create spacy training data
train_data = json_to_spacy('../input/ner-annotation-of-city-of-la-jobs/city_la_jobs_ner_labeling.json')

## Train NER model

In [6]:
import random
import spacy

# Model training function
def ner_model(train_data, n_iter=50):
    
    # Create Ner model and add labels
    nlp = spacy.blank("en")
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    # Begin training
    nlp.begin_training()
    
    for itn in range(n_iter):
        random.shuffle(train_data)
        losses = {}
        batches = spacy.util.minibatch(train_data, size=spacy.util.compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts,annotations,drop=0.5,losses=losses)
        if itn%10==0:
            print(losses)
    return nlp

nlp = ner_model(train_data)
nlp.to_disk('./')

{'ner': 5367.079756039195}
{'ner': 3404.516049408646}
{'ner': 1993.4781608458325}
{'ner': 1909.0222931467833}
{'ner': 1862.9048940600162}


In [7]:
# Test the model with predictions
for text, _ in train_data:
    doc = nlp(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('Four years', 'EXPERIENCE_LENGTH'), ('full-time', 'FULL_TIME_PART_TIME'), ('City of Los Angeles', 'EXP_JOB_COMPANY'), ('collection or disposal of municipal wastes', 'EXP_JOB_CLASS_FUNCTION')]
Tokens [('Four', 'EXPERIENCE_LENGTH', 3), ('years', 'EXPERIENCE_LENGTH', 1), ('of', '', 2), ('full', 'FULL_TIME_PART_TIME', 3), ('-', 'FULL_TIME_PART_TIME', 1), ('time', 'FULL_TIME_PART_TIME', 1), ('paid', '', 2), ('experience', '', 2), ('with', '', 2), ('the', '', 2), ('City', 'EXP_JOB_COMPANY', 3), ('of', 'EXP_JOB_COMPANY', 1), ('Los', 'EXP_JOB_COMPANY', 1), ('Angeles', 'EXP_JOB_COMPANY', 1), ('in', '', 2), ('connection', '', 2), ('with', '', 2), ('the', '', 2), ('collection', 'EXP_JOB_CLASS_FUNCTION', 3), ('or', 'EXP_JOB_CLASS_FUNCTION', 1), ('disposal', 'EXP_JOB_CLASS_FUNCTION', 1), ('of', 'EXP_JOB_CLASS_FUNCTION', 1), ('municipal', 'EXP_JOB_CLASS_FUNCTION', 1), ('wastes', 'EXP_JOB_CLASS_FUNCTION', 1), ('.', '', 2), ('Completion', '', 2), ('of', '', 2), ('30', '', 2), ('semester', '

## Extract using NER model

In [8]:
# Extract data by ner model
def ner_extract(nlp,text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

# Extract requirements 
df['temp_req'] = df['req_list'].apply(lambda x: ner_extract(nlp,x))

In [9]:
df.head(5)

,File_Name,job_info,JOB_CLASS_TITLE,JOB_CLASS_NO,OPEN_DATE,EXAM_TYPE,ENTRY_SALARY_GEN,ENTRY_SALARY_DWP,JOB_DUTIES,DRIVERS_LICENSE_REQ,DRIV_LIC_TYPE,Req_section,req_list,REQUIREMENT_SET_ID,temp_req
0,PARK MAINTENANCE SUPERVISOR 3145 102618.txt,PARK MAINTENANCE SUPERVISOR\n\nClass Code: ...,PARK MAINTENANCE SUPERVISOR,3145,10-26-18,OPEN,None,"83,728 - 88,405",A Park Maintenance Supervisor directly supervi...,R,None,1. Two years of full-time paid experience as a...,1. Two years of full-time paid experience as a...,1,"[(Two years, EXPERIENCE_LENGTH), (full-time, F..."
1,PARK MAINTENANCE SUPERVISOR 3145 102618.txt,PARK MAINTENANCE SUPERVISOR\n\nClass Code: ...,PARK MAINTENANCE SUPERVISOR,3145,10-26-18,OPEN,None,"83,728 - 88,405",A Park Maintenance Supervisor directly supervi...,R,None,1. Two years of full-time paid experience as a...,2. One year full-time paid experience as noted...,2,"[(One year, EXPERIENCE_LENGTH), (full-time, FU..."
2,ENVIRONMENTAL ENGINEER 7872 082616 REV 090116...,ENVIRONMENTAL ENGINEER \n\nClass Code: 7...,ENVIRONMENTAL ENGINEER,7872,08-26-16,OPEN,"89,637 - 127,555",None,An Environmental Engineer performs responsible...,R,None,Two years of full-time paid professional envir...,Two years of full-time paid professional envir...,1,"[(Two years, EXPERIENCE_LENGTH), (full-time, F..."
3,MOTION PICTURE AND TELEVISION MANAGER 1789 111...,MOTION PICTURE AND TELEVISION MANAGER\n\n\nCla...,MOTION PICTURE AND TELEVISION MANAGER,1789,11-17-17,OPEN,"120,080 - 175,559",None,"A Motion Picture and Television Manager plans,...",R,None,1. Graduation from an accredited four-year col...,1. Graduation from an accredited four-year col...,1,"[(four-year, EDUCATION_YEARS), (college or uni..."
4,HOUSING INVESTIGATOR 8516 062918.txt,HOUSING INVESTIGATOR\n\nClass Code: 8516...,HOUSING INVESTIGATOR,8516,06-29-18,OPEN,"59,487 - 86,965",None,A Housing Investigator conducts preliminary an...,R,None,1. Graduation from an accredited four-year col...,1. Graduation from an accredited four-year col...,1,"[(four-year, EDUCATION_YEARS), (college or uni..."


# Create CSV

##  Flatten the temp_req column

In [10]:
# Extract one column from list
def extract_col(l,col):
    result = [x[0] for x in l if x[1]==col]
    return None if result==[] else result[0]

# Extract colums from temp_req column
def extract_col_df(df,col):
    df[col]=df['temp_req'].apply(lambda x: extract_col(x,col))
    return df


In [11]:
# Extract coulums in df
cols_to_extract = ['EDUCATION_YEARS', 'SCHOOL_TYPE' ,'EDUCATION_MAJOR', 'EXPERIENCE_LENGTH' ,\
                   'FULL_TIME_PART_TIME','EXP_JOB_CLASS_TITLE', 'EXP_JOB_CLASS_ALT_RESP',\
                   'EXP_JOB_CLASS_FUNCTION', 'EXP_JOB_CLASS_ADDITIONAL_FUNCTION','COURSE_COUNT',\
                   'COURSE_LENGTH', 'COURSE_SUBJECT', 'MISC_COURSE_DETAILS', 'EXP_JOB_COMPANY',\
                   'DEGREE NAME', 'EXP_JOB_CLASS_ALT_JOB_TITLE', 'REQUIRED_CERTIFICATE',\
                   'CERTIFICATE_ISSUED_BY','COURSE_TITLE', 'REQUIRED_EXAM_PASS', 'EXPERIENCE_EXTRA_DETAILS']

for col in cols_to_extract:
    df = extract_col_df(df,col)

In [12]:
col_orders = ['File_Name','JOB_CLASS_TITLE', 'JOB_CLASS_NO', 
                 'REQUIREMENT_SET_ID',
                 'JOB_DUTIES','EDUCATION_YEARS', 'SCHOOL_TYPE',
                 'EDUCATION_MAJOR', 'DEGREE NAME','EXPERIENCE_LENGTH',
                 'FULL_TIME_PART_TIME',
                 'EXP_JOB_CLASS_TITLE', 'EXP_JOB_CLASS_FUNCTION',
                 'EXP_JOB_COMPANY','EXP_JOB_CLASS_ALT_JOB_TITLE',
                 'EXP_JOB_CLASS_ALT_RESP', 'COURSE_COUNT',
                 'COURSE_LENGTH', 'COURSE_SUBJECT',
                 'REQUIRED_CERTIFICATE','CERTIFICATE_ISSUED_BY',
                 'DRIVERS_LICENSE_REQ', 'DRIV_LIC_TYPE',
                 'EXAM_TYPE','ENTRY_SALARY_GEN','ENTRY_SALARY_DWP','OPEN_DATE']

df[col_orders].to_csv('./jobs.csv', index=None)